In [3]:
!pip install datasets
from datasets import load_dataset 

In [4]:
pubmed = load_dataset(
   'pubmed_qa',
   'pqa_labeled',
   split='train'
)
pubmed

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

In [5]:
contexts = []
for record in pubmed['context']:
    contexts.append('\n'.join(record['contexts']))
for context in contexts[:3]:
    print(f"{context[:300]}...")

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cel...
Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differenc...
Apparent life-threatening events in infants are a difficult and frequent problem in pediatric practice. The prognosis is uncertain because of risk of sudden infant death syndrome.
Eight infants aged 2 to 15 months were admitted during a period of 6 years; they suffered from similar maladies in the b...


In [6]:
# Sparse Vector
!pip install transformers
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(
   'bert-base-uncased'
)

In [7]:
from collections import Counter

def build_dict(input_batch):
    sparse_emb = []
    for token_ids in input_batch:
        indices = []
        values = []
        d = dict(Counter(token_ids))
        for indice in d:
            indices.append(indice)
            values.append(d[indice])
        sparse_emb.append({'indices': indices, 'values': values})
    return sparse_emb

def generate_sparse_vectors(context_batch):
    inputs = tokenizer(
           context_batch, padding=True,
           truncation=True,
           max_length=512
    )['input_ids']
    sparse_embeds = build_dict(inputs)
    return sparse_embeds

contexts = []
for record in pubmed['context']:
    contexts.append('\n'.join(record['contexts']))

sparse_vectors = generate_sparse_vectors(contexts)


In [8]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi
from collections import Counter
import numpy as np

def build_bm25(input_batch):
    # Initialize BM25
    bm25 = BM25Okapi(input_batch)
    
    # Create sparse embeddings using BM25 scores
    sparse_emb = []
    for token_ids in input_batch:
        indices = []
        values = []
        # Get BM25 scores for the current document
        scores = bm25.get_scores(token_ids)
        for idx, score in enumerate(scores):
            if score > 0:  # Only consider non-zero scores
                indices.append(idx)
                values.append(score)
        sparse_emb.append({'indices': indices, 'values': values})
    return sparse_emb

def generate_sparse_vectors_bm25(context_batch):
    # Tokenize input text
    inputs = tokenizer(
        context_batch, padding=True,
        truncation=True, max_length=512
    )['input_ids']
    
    # Convert inputs to string format as BM25 requires text input
    inputs_str = [[str(token) for token in doc] for doc in inputs]
    
    # Create sparse dictionaries using BM25
    sparse_embeds = build_bm25(inputs_str)
    return sparse_embeds

# Example usage
contexts = []
for record in pubmed['context']:
    contexts.append('\n'.join(record['contexts']))

sparse_vectors_bm25 = generate_sparse_vectors_bm25(contexts)


In [9]:
print(f"First sparse embedding: {sparse_vectors_bm25[0]}")

First sparse embedding: {'indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214

In [10]:
# Install the required libraries
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

emb = model.encode(contexts[0])

print(emb.shape)



(384,)


In [11]:
# Install the Pinecone client
!pip install pinecone-client

# Import necessary modules
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize the Pinecone instance
pc = Pinecone(
    api_key="cc72d653-271c-42ed-bc71-4209c9409e6b" 
)

# Define the index name
index_name = "hybrid-search"

# Check if the index already exists
if index_name not in pc.list_indexes().names():
    # Create the index
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimensionality of dense model
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )


In [14]:
from tqdm.auto import tqdm
import pinecone
index = pinecone.Index(
    name="hybrid-search",
    host="https://hybrid-search-okle6cz.svc.aped-4627-b74a.pinecone.io",
    api_key="cc72d653-271c-42ed-bc71-4209c9409e6b"
)
batch_size = 32
for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    end_batch = min(i + batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:end_batch]
    # create unique IDs
    unique_ids = [str(id) for id in range(i, end_batch)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vector = []
    for _id, sparse, dense, metadata in zip(unique_ids, sparse_embeds, dense_embeds, meta):
        sparse_values = [float(value) for value in sparse['values']]
        vector.append(
            {
                'id': _id,
                'values': dense,
                'sparse_values': {
                    'indices': sparse['indices'],
                    'values': sparse_values
                },
                'metadata': metadata
            }
        )
    index.upsert(vector)

index.describe_index_stats()


  0%|          | 0/32 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [20]:
def hybrid_scale(dense, sparse, alpha: float):
    if alpha < 0 or alpha > 1:
        raise ValueError("Alpha must between 0 and 1")

    hsparse = {
        'indices': sparse['indices'],
        'values': [x*(1-alpha) for x in sparse['values']]
    }
    if isinstance(dense[0], list):
        dense = dense[0]
    hdense = [x*alpha for x in dense]
    return hsparse, hdense

def hybrid_query(question, top_k, alpha):
    sparse_vec = generate_sparse_vectors([question])[0]
    #print("Sparse Vector:", sparse_vec)
    
    dense_vec = model.encode([question]).tolist()
    #print("Dense Vector:", dense_vec)
    

    sparse_vec, dense_vec = hybrid_scale(dense_vec, sparse_vec, alpha)
    # print("Scaled Dense Vector:", dense_vec)
    # print("Scaled Sparse Vector:", sparse_vec)
    
    result = index.query(
        vector = dense_vec,
        sparse_values = sparse_vec,
        top_k = top_k,
        include_metadata = True
    )
    return result

In [26]:
question = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"
hybrid_query(question, top_k=3, alpha=0)

{'matches': [{'id': '994',
              'metadata': {'context': 'Although desflurane is commonly used to '
                                      'control surgically induced '
                                      'hypertension, its effects on left '
                                      'ventricular (LV) function have not been '
                                      'investigated in this clinical '
                                      'situation. The purpose of the present '
                                      'study was to evaluate the LV function '
                                      'response to desflurane, when used to '
                                      'control intraoperative hypertension.\n'
                                      'In 50 patients, scheduled for vascular '
                                      'surgery, anesthesia was induced with '
                                      'sufentanil 0.5 microg/kg, midazolam 0.3 '
                                      'mg/k